In [3]:
import re
from datetime import datetime
from pathlib import Path
import os
sc = pyspark.SparkConf()


In [73]:
import findspark as fs
fs.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField, ArrayType, FloatType
from pyspark.sql.functions import udf, from_json, col
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import PipelineModel
from delta import *

In [28]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0 pyspark-shell'
# init spark here


In [74]:
schema = StructType(
    [StructField("created_at", StringType()),
    StructField("message", StringType()),
    StructField("City", StringType()),
    StructField("Temp", FloatType())]
    )


In [75]:
spark = SparkSession \
        .builder.appName("test") \
        .master("local[*]") \
        .config("spark.streaming.stopGracefullyOnShutdown", 
     "true") \
        .config("spark.jars.packages”,”org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
        .getOrCreate()


In [76]:
kafka_topic = 'test'
ml_path = ""

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "latest") \
  .load()\
  .selectExpr("CAST(timestamp AS TIMESTAMP) as timestamp", "CAST(value AS STRING) as message")
df = df \
    .withColumn("value", from_json("message", schema)) \
    .select('timestamp', 'value.*')
print(df)


DataFrame[timestamp: timestamp, created_at: string, message: string, City: string, Temp: float]


In [139]:
import re
S= 'Salman, Dahir, Salman'
try:
    d=re.search('^[\w ]+,', S)
    print (d.group())
except:
    print (S)



Salman,


In [26]:
schema = StructType(
    [StructField("created_at", StringType()),
    StructField("message", StringType())]
    )
df = df \
    .withColumn("value", from_json("message", schema)) \
    .select('timestamp', 'value.*')

In [27]:
import pyowm
import json
import datetime
api_key = '3d2e4262254bb8b94eaf837891e9bc38'    #your API Key here as string
owm = pyowm.OWM(api_key).weather_manager()

def get_current_weather():
    try:    
        weather_api = owm.weather_at_place('Los Angeles,')  # give where you need to see the weather
        weather_data = weather_api.weather
        print(type(weather_data))          # get out data in the mentioned location
        print("***Current Weather***")
        wdict = weather_data.temperature('celsius')
        avg_temp = (wdict['temp_max'] + wdict['temp_min'])/2
        print(avg_temp)
    except: 
        print ('null')
    print("\n")


get_current_weather() 


<class 'pyowm.weatherapi25.weather.Weather'>
***Current Weather***
19.6




In [40]:
import pandas as pd

[{
    "timestamp": "Tue Jun 21 16:17:17 +0000 2022","created_at": "Tue Jun 21 16:17:17 +0000 2022", "message": "⚠️ How many parents have received more than one text message from Texas Childrens Hospital regarding the child COVID vaccine? \n\nI’ve spoken with several people who have received over 6 text messages since Tuesday. This seems highly unusual.", "City": "None", "Temp": "null"},
{"timestamp": "Tue Jun 21 16:17:17 +0000 2022", "created_at": "Tue Jun 21 16:17:17 +0000 2022", "message": "@DiscordDystopia I started publicly showing my art right before COVID struck but I'd really like to do more live i… https://t.co/lXahOUobDD", "City": "Ohio, US", "Temp": "23.5"}
]

[{'timestamp': 'Tue Jun 21 16:17:17 +0000 2022',
  'created_at': 'Tue Jun 21 16:17:17 +0000 2022',
  'message': '⚠️ How many parents have received more than one text message from Texas Childrens Hospital regarding the child COVID vaccine? \n\nI’ve spoken with several people who have received over 6 text messages since Tuesday. This seems highly unusual.',
  'City': 'None',
  'Temp': 'null'},
 {'timestamp': 'Tue Jun 21 16:17:17 +0000 2022',
  'created_at': 'Tue Jun 21 16:17:17 +0000 2022',
  'message': "@DiscordDystopia I started publicly showing my art right before COVID struck but I'd really like to do more live i… https://t.co/lXahOUobDD",
  'City': 'Ohio, US',
  'Temp': '23.5'}]

In [ ]:
val query = source.writeStream \
  .option("checkpointLocation", checkpointDir.toString)\
  .format("org.apache.spark.sql.cassandra")\
    .option("keyspace", ks)\
    .option("table", "kv")\
  .outputMode(OutputMode.Update)
  .start()

In [39]:
from pyspark.sql.functions import udf, from_json, col
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField, ArrayType, FloatType
from pyspark.sql.functions import udf, from_json, col
import findspark
from datetime import datetime
spark = SparkSession \
    .builder \
    .master("local[*]")\
    .appName("StreamingApp") \
    .getOrCreate()

findspark.init()

schema = StructType(
    [StructField("created_at", StringType()),
    StructField("message", StringType()),
    StructField("City", StringType()),
    StructField("Temp", FloatType())]
    )

df = spark.read.json("/Users/Salman/Desktop/Github/Kafka-Spark-Twitter-Weather-Streaming/twitter/sample2.json")

df.show()

df = df \
    .withColumn("value", from_json("message", schema)) 
  
#print(df)
#Changing datetime format

date_process = udf(
    lambda x: datetime.strftime(
        datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S'
        )
    )
df = df.withColumn("created_at", date_process(df.created_at))
df.show()

+--------------------+----+--------------------+--------------------+--------------------+
|                City|Temp|     _corrupt_record|          created_at|             message|
+--------------------+----+--------------------+--------------------+--------------------+
|                null|null|                  [{|                null|                null|
|                null|null|"created_at": "Tu...|                null|                null|
|abstract.the.orig...|null|                null|Tue Jun 21 16:17:...|@DiscordDystopia ...|
|                null|null|                   ]|                null|                null|
+--------------------+----+--------------------+--------------------+--------------------+

22/06/22 11:09:32 ERROR Executor: Exception in task 0.0 in stage 61.0 (TID 61)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/var/folders/11/1_98cps925d273zrhlp6wcs40000gn/T/ipykernel_635/3258052198.py", line 34, in <lambda>
Type

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/var/folders/11/1_98cps925d273zrhlp6wcs40000gn/T/ipykernel_635/3258052198.py", line 34, in <lambda>
TypeError: strptime() argument 1 must be str, not None
